In [14]:
import requests
import json
import time
import pandas as pd
from IPython.display import clear_output

loop_count = 250
beer_count = 0

beerDF = pd.DataFrame(columns=['id','name','abv','ph','ibu'])

for x in range(230, loop_count+1):
    #Use IPython custome clear output to minimise outputs
    clear_output(wait=True)
    
    #Get JSON using URL Request
    url_api = 'https://api.punkapi.com/v2/beers/' + str(x)
    response = requests.get(url=url_api, headers = {'Content-Type': 'application/json', 'Accept': 'application/json'})
    
    #If error, API will return differently structured JSON rather than a HTTP error
    try:
        #Store JSON
        json_beer = json.loads(response.text)[0]
    except:
        #Error - so skip (but show ID with error)
        print("Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far! No beer with ID: ",x)
        continue
        
    #Assign beer data into Dictionary for later analysis
    beer_list = {}
    beer_list['id'] = json_beer['id']
    beer_list['name'] = json_beer['name']
    beer_list['abv'] = json_beer['abv']
    beer_list['ph'] = json_beer['ph']
    beer_list['ibu'] = json_beer['ibu']
    
    #Append to dataframe
    beerDF.append(pd.Series(beer_list), ignore_index=True)
    df  = pd.DataFrame([beer_list], columns=beer_list.keys())
    
    #Add beer count
    beer_count  += 1
    
    #Print progress
    print("Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far!")
    
    #Wait fixed time to prevent API throttling / restricting usage
    time.sleep(.75)

Current progress: 250 / 250 .  5  beer(s) discovered so far! No beer with ID:  250


In [15]:
beerDF

,id,name,abv,ph,ibu
